Loading the data

id: household

Choices:  
- heinz41  
- heinz32  
- heinz28  
- hunts32  

variables  
- disp (1/0)  
- feat (1/0)  
- price (Log?)  

In [1]:
import numpy as np
import pandas as pd
df = pd.read_pickle("data.pkl") 
df.head()
# choices = ['heinz41', 'heinz32', 'heinz28', 'hunts32']
# df['choiceindex'] = list(map(lambda x: choices.index(x), df.choice))


,id,disp.heinz41,disp.heinz32,disp.heinz28,disp.hunts32,feat.heinz41,feat.heinz32,feat.heinz28,feat.hunts32,price.heinz41,price.heinz32,price.heinz28,price.hunts32,choice,choiceindex
0,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28,2
1,1,0,0,0,0,0,0,0,0,4.6,4.3,5.2,4.4,heinz28,2
2,1,0,0,0,0,0,1,0,0,4.6,2.5,4.6,4.8,heinz28,2
3,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28,2
4,1,0,0,0,0,0,0,1,0,4.6,3.0,4.6,4.8,heinz28,2


Defining the likelihood function using numba and numpy arrays and multiple draws

In [3]:
from numba import jit, prange

@jit(nopython=True, parallel=True)
def likelihood(c, data, draws):
    n_r = draws.shape[0]
    n_q = draws.shape[1]
#   n_k = draws.shape[2]
    
    #matrix with all simulations
    simulations = np.zeros((n_q, n_r))
    
    #iterate over households
    for q in prange(n_q):
        rows = data[np.where(data[:,0] == q+1)]
        n_rows = len(rows)
        
        #iterate over draws per household
        for r in prange(n_r):
            probabilities= np.zeros(n_rows)
            
            #iterate over oberservations per househould
            for t in prange(n_rows):
                choices = np.zeros(4)
                
                #itetate over probability of choices per observation
                for j in prange(4):
                    utility = 0; #start with alpha
                    if j < 3: utility = c[j]
                    x = [rows[t][1+j], rows[t][5+j], rows[t][9+j]]
                    mu =  c[3:6]
                    sigma = c[6:]
                    #np.exp(alpha +  np.dot(c[3:6],x) + np.dot(np.multiply(c[6:],draws[row[0]-1]),x))
                    for l in prange(3):
                          utility += mu[l] * x[l] + sigma[l] * draws[r][q][l] * x[l]

                    choices[j] = np.exp(utility)
            
                probabilities[t] = choices[int(rows[t][13])] / np.sum(choices)
                
            simulations[q,r] = np.exp(np.log(probabilities).sum())
            
    estimates = np.zeros(n_q)
    for q in prange(n_q):
        estimates[q] = np.sum(simulations[q,:]) / n_r
    
    return np.log(estimates).sum()



benchmarking and comparing pandas and numba implementation

In [4]:
np.random.seed(1234)
n = 10
draws = np.random.randn(10, 300, 3); 

coefficients = [1, #alpha heinz41
                3, #alpha heinz32 
                2, #alpha hunts32
                2, #mu    display
                4, #mu    feat
                8, #mu    price
                3, #sigma dispay 
                1, #sigma feat 
                2, #sigma price 
                ]
print(likelihood(coefficients, df.drop(columns='choice').values, draws))


# # DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
# start = time.time()
# print(likelihood(coefficients, df.drop(columns='choice').values, draws))
# end = time.time()
# print("Elapsed (first run) = %s" % (end - start))

# # NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
# start = time.time()
# for i in range(n):
#     likelihood(coefficients, df.drop(columns='choice').values, draws)
# end = time.time()
# print("Elapsed (compiled) for %d iterations = %s" % (n, (end - start)))
# print((end - start)/n)

UnsupportedError: Failed in nopython mode pipeline (step: nopython mode backend)
[1mFailed in nopython mode pipeline (step: nopython frontend)
[1mUnsupported op-code encountered: arrayexpr(expr=(<built-in function eq>, [Var($50.12, <ipython-input-3-4d292045aa72> (14)), Var($50.15, <ipython-input-3-4d292045aa72> (14))]), ty=array(bool, 1d, F))
[1m
File "<ipython-input-3-4d292045aa72>", line 14:[0m
[1mdef likelihood(c, data, draws):
    <source elided>
    for q in prange(n_q):
[1m        rows = data[np.where(data[:,0] == q+1)]
[0m        [1m^[0m[0m
[0m
[0m[1m[1] During: lowering "id=15[LoopNest(index_variable = parfor_index.205, range = (0, $n_q.207, 1))]{50: <ir.Block at <ipython-input-3-4d292045aa72> (13)>}Var(parfor_index.205, <ipython-input-3-4d292045aa72> (13))" at <ipython-input-3-4d292045aa72> (13)[0m
Unsupported functionality was found in the code Numba was trying to compile.

If this functionality is important to you please file a feature request at:
https://githu

In [434]:
np.random.seed(1234)
opfd = np.random.rand(3,5)

print(opfd)

print(opfd[0,:].sum() / 5)

print(np.mean(opfd, axis=1))

len([0,1,2])
draws = np.random.randn(10, 4, 3); 
len(draws[1])

for i in range(4):
    print(i)

[[0.19151945 0.62210877 0.43772774 0.78535858 0.77997581]
 [0.27259261 0.27646426 0.80187218 0.95813935 0.87593263]
 [0.35781727 0.50099513 0.68346294 0.71270203 0.37025075]]
0.5633380704516823
[0.56333807 0.63700021 0.52504562]
0
1
2
3
